<a href="https://colab.research.google.com/github/zzaeyun/ESAA23_1/blob/main/%EC%95%99%EC%83%81%EB%B8%94_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___


직접 투표 hard voting: 각 분류기의 결과를 사용하여 가장 많이 나온 예측값을 최종 예측값으로 설정 (다수결)

간접 투표 soft voting: 각 분류기 별로 예측값에 대한 확률 (proba) 을 구해서 평균과 같은 방법으로 모든 분류기의 proba를 합친 후 예측값을 결정

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

학습률을 낮게 설정해야 한다.


### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [3]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

/usr/local/lib/python3.9/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [9]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다. 

In [5]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [6]:
# model fitting

rf_clf=RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

etc_clf=ExtraTreesClassifier(n_estimators=100, random_state=42)
etc_clf.fit(X_train, y_train)

svc_clf=LinearSVC(random_state=42)
svc_clf.fit(X_train, y_train)

mlp_clf=MLPClassifier( random_state=42)
mlp_clf.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


MLPClassifier(random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [7]:
from sklearn.ensemble import VotingClassifier

In [ ]:
# model fitting
voting_clf=VotingClassifier(
    estimators=[('rf', rf_clf), ('etc', etc_clf), ('svc', svc_clf), ('mlp', mlp_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('etc', ExtraTreesClassifier(random_state=42)),
                             ('svc', LinearSVC(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [ ]:
# model test
voting_clf.score(X_test, y_test)

0.9692571428571428

####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [ ]:
# 각 분류 모델 학습


In [ ]:
# 각 분류 모델의 성능 확인

from sklearn.metrics import accuracy_score

rf_pred=rf_clf.predict(X_test)
print('rf',accuracy_score(y_test, rf_pred))

etc_pred=etc_clf.predict(X_test)
print('etc',accuracy_score(y_test, etc_pred))

svc_pred=svc_clf.predict(X_test)
print('svc',accuracy_score(y_test, svc_pred))

mlp_pred=mlp_clf.predict(X_test)
print('mlp',accuracy_score(y_test, mlp_pred))

rf 0.9673142857142857
etc 0.9716
svc 0.8545142857142857
mlp 0.9666285714285714


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. SVM

In [ ]:
# 가장 성능이 낮은 모델 제거

voting2_clf=VotingClassifier(
    estimators=[('rf', rf_clf), ('etc', etc_clf), ('mlp', mlp_clf)],
    voting='hard'
)

In [ ]:
# model fitting

voting2_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('etc', ExtraTreesClassifier(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [ ]:
# 모델 제거 후 성능 확인

voting2_clf.score(X_test, y_test)

0.9732

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [10]:
estimators = [rf_clf, etc_clf, svc_clf, mlp_clf]

In [11]:
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

In [12]:
for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [14]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, random_state=42)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다. 

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [15]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [16]:
# 새로운 데이터셋을 이용하여 블렌더로 예측

y_pred = rnd_forest_blender.predict(X_test_predictions)

In [17]:
# model test
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9923